In [6]:
import pandas as pd
pd.set_option('display.max_columnss', 500)

In [16]:
data = pd.read_csv('data/music_data_merged_spotify.csv', low_memory=False, index_col=False)
data.drop('Unnamed: 0', axis = 1, inplace = True)

In [24]:
data[~(data.spotify_id.isna()) & ~(data.spotify_album_id.isna())]

,Statement Month,Store Name,Label Name,Artist Name,Release Name,Track Artist (Performer),Track Name,ISRC,Display Upc,Country Code,Continent,"Stream Ad-Supp, Stream Prem, Download, Physical, Locker, Other",Units,Gross Revenue USD,Net Revenue USD,spotify_id,spotify_album_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,release_date,release_date_precision,total_tracks,type
7,2018-01,Spotify,MRI Entertainment,Rebelution,Count Me In,Rebelution,Fade Away,US4CL1410008,886444576840,US,North America,Stream Premium,"712,780","$3,510.91","$3,159.82",4TyCnstYu6LGrjka5WW6ft,05SuSxEPuEJI1RnwTcOCRg,0.685,0.540,9.0,-9.701,1.0,0.0253,0.1810,0.015000,0.1140,0.235,109.926,209253.0,4.0,2014-06-10,day,11.0,album
8,2021-03,Pandora,MRI Entertainment,Rebelution,Count Me In,Rebelution,Roots Reggae Music (feat. Don Carlos),US4CL1410011,886444576840,US,North America,Stream Premium,"280,323","$3,373.37","$3,036.04",49CdYBpfABUa0ZfT8FizQZ,05SuSxEPuEJI1RnwTcOCRg,0.920,0.560,8.0,-9.530,1.0,0.0576,0.0551,0.000041,0.0970,0.625,140.004,273996.0,4.0,2014-06-10,day,11.0,album
9,2021-03,Pandora,MRI Entertainment,Rebelution,Count Me In,Rebelution,Roots Reggae Music (feat. Don Carlos),US4CL1410011,886444576840,US,North America,Stream Ad-Supported,"635,968","$3,223.90","$2,901.51",49CdYBpfABUa0ZfT8FizQZ,05SuSxEPuEJI1RnwTcOCRg,0.920,0.560,8.0,-9.530,1.0,0.0576,0.0551,0.000041,0.0970,0.625,140.004,273996.0,4.0,2014-06-10,day,11.0,album
10,2019-08,Spotify,MRI Entertainment,Rebelution,Count Me In,Rebelution,Fade Away,US4CL1410008,886444576840,US,North America,Stream Premium,"687,480","$3,110.93","$2,799.84",4TyCnstYu6LGrjka5WW6ft,05SuSxEPuEJI1RnwTcOCRg,0.685,0.540,9.0,-9.701,1.0,0.0253,0.1810,0.015000,0.1140,0.235,109.926,209253.0,4.0,2014-06-10,day,11.0,album
11,2021-02,Pandora,MRI Entertainment,Rebelution,Count Me In,Rebelution,Roots Reggae Music (feat. Don Carlos),US4CL1410011,886444576840,US,North America,Stream Premium,"282,556","$3,063.66","$2,757.29",49CdYBpfABUa0ZfT8FizQZ,05SuSxEPuEJI1RnwTcOCRg,0.920,0.560,8.0,-9.530,1.0,0.0576,0.0551,0.000041,0.0970,0.625,140.004,273996.0,4.0,2014-06-10,day,11.0,album
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4757925,2021-07,iTunes/Apple,MRI Entertainment,Rebelution,In the Moment,Rebelution,Old School Feeling,US4CL2110025,657481109182,US,North America,Download,-572,-$520.52,-$468.47,54GuCaNBEr3dcPAZq4SNkB,4ZO52CTQ2EzwDmAIrFjHaW,0.975,0.624,11.0,-6.891,0.0,0.0512,0.0373,0.002540,0.0831,0.965,125.049,236625.0,4.0,2021-06-18,day,15.0,album
4757926,2018-07,iTunes/Apple,MRI Entertainment,Rebelution,Free Rein,Rebelution,Patience,US4CL1810006,657481106983,US,North America,Download,"-2,013","-$1,831.83","-$1,648.65",0y6EUHrKgUIEc0yqGg4acO,5FfbqVo0OapeCvoJvjWlOp,0.793,0.495,7.0,-8.631,0.0,0.0367,0.0379,0.000027,0.1230,0.964,134.920,206222.0,4.0,2018-06-15,day,12.0,album
4757927,2018-07,iTunes/Apple,MRI Entertainment,Rebelution,Free Rein,Rebelution,Celebrate,US4CL1810001,657481106983,US,North America,Download,"-2,308","-$2,100.28","-$1,890.25",26IEVYmeFletNA10cOkdBb,5FfbqVo0OapeCvoJvjWlOp,0.660,0.679,2.0,-6.749,1.0,0.1110,0.0224,0.000000,0.0945,0.595,164.110,243174.0,4.0,2018-06-15,day,12.0,album
4757928,2018-07,iTunes/Apple,MRI Entertainment,Rebelution,Free Rein,Rebelution,City Life,US4CL1810002,657481106983,US,North America,Download,"-2,586","-$2,353.26","-$2,117.93",1ozK6iAeBgCOSXecRojzMo,5FfbqVo0OapeCvoJvjWlOp,0.691,0.845,7.0,-4.746,0.0,0.0519,0.0215,0.000013,0.3310,0.635,162.062,207648.0,4.0,2018-06-15,day,12.0,album


In [17]:
tracks_data = data[data.type == 'track']

In [18]:
len(tracks_data.ISRC.unique())

177

In [19]:
tracks_data.head()

,Statement Month,Store Name,Label Name,Artist Name,Release Name,Track Artist (Performer),Track Name,ISRC,Display Upc,Country Code,Continent,"Stream Ad-Supp, Stream Prem, Download, Physical, Locker, Other",Units,Gross Revenue USD,Net Revenue USD,spotify_id,spotify_album_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,release_date,release_date_precision,total_tracks,type
482,2021-06,Peloton,MRI Entertainment,The Green,Hawai'i '13,The Green,Power in the Words,US4CL1310031,886444083997,US,North America,Stream Premium,"48,250","$1,051.81",$946.63,7HIV5qR08KfnTL3lWazcwa,NaN,0.744,0.756,10.0,-8.107,0.0,0.0427,0.06870,0.000004,0.117,0.906,87.992,278333.0,4.0,NaN,NaN,NaN,track
1015,2020-07,Peloton,MRI Entertainment,The Green,Hawai'i '13,The Green,Hawai'i Aloha,US4CL1310040,886444083997,US,North America,Stream Premium,"21,873",$725.90,$653.31,4qyyWB9ah6Rd6eMmRsR26q,NaN,0.193,0.467,11.0,-14.405,1.0,0.0458,0.79600,0.000000,0.443,0.314,112.450,110431.0,3.0,NaN,NaN,NaN,track
1389,2018-04,iTunes/Apple,MRI Entertainment,Protoje,Bout Noon,Protoje,Bout Noon,US4CL1810013,657481304280,US,North America,Stream Premium,"72,127",$588.99,$530.09,1mO183DNefKbb43wp4G7PN,NaN,0.667,0.777,10.0,-5.261,0.0,0.2430,0.00402,0.000000,0.126,0.691,83.899,224328.0,4.0,NaN,NaN,NaN,track
1645,2021-07,Peloton,MRI Entertainment,The Green,Hawai'i '13,The Green,Take Me On,US4CL1310026,886444083997,US,North America,Stream Premium,"21,839",$523.40,$471.06,31LJJKqdYLg1iMc68zddDu,NaN,0.774,0.544,9.0,-9.920,1.0,0.0921,0.04720,0.000000,0.115,0.965,145.022,204867.0,4.0,NaN,NaN,NaN,track
1696,2019-08,iTunes/Apple,MRI Entertainment,Lila Iké,Where I'm Coming From,Lila Iké,Where I'm Coming From,US4CL1910036,657481306222,US,North America,Stream Premium,"69,474",$511.38,$460.24,2jQENRPVxXb8TcTAXW6wpQ,NaN,0.500,0.698,1.0,-3.878,0.0,0.3840,0.44200,0.000000,0.683,0.730,93.035,243050.0,5.0,NaN,NaN,NaN,track


In [23]:
tracks_data[tracks_data['Statement Month'] == '2021-08']

,Statement Month,Store Name,Label Name,Artist Name,Release Name,Track Artist (Performer),Track Name,ISRC,Display Upc,Country Code,Continent,"Stream Ad-Supp, Stream Prem, Download, Physical, Locker, Other",Units,Gross Revenue USD,Net Revenue USD,spotify_id,spotify_album_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,release_date,release_date_precision,total_tracks,type
3546,2021-08,iTunes/Apple,MRI Entertainment,The Green,Hawai'i '13,The Green,Good One,US4CL1310030,886444083997,US,North America,Stream Premium,"39,962",$267.63,$240.86,0RVuF9ekXb1mih0exuh4ZQ,NaN,0.881,0.573,2.0,-8.067,1.0,0.1050,0.04540,0.000000,0.1280,0.925,130.025,206000.0,4.0,NaN,NaN,NaN,track
4025,2021-08,iTunes/Apple,MRI Entertainment,The Green,Hawai'i '13,The Green,Chocolates & Roses,US4CL1310033,886444083997,US,North America,Stream Premium,"35,202",$232.83,$209.54,2uEfZM9njlfBSOrFUrlq6V,NaN,0.562,0.298,0.0,-11.853,1.0,0.0481,0.15300,0.000000,0.0939,0.308,131.336,303013.0,4.0,NaN,NaN,NaN,track
5808,2021-08,Spotify,MRI Entertainment,The Green,Hawai'i '13,The Green,Chocolates & Roses,US4CL1310033,886444083997,US,North America,Stream Premium,"35,060",$161.35,$145.22,2uEfZM9njlfBSOrFUrlq6V,NaN,0.562,0.298,0.0,-11.853,1.0,0.0481,0.15300,0.000000,0.0939,0.308,131.336,303013.0,4.0,NaN,NaN,NaN,track
9313,2021-08,Spotify,MRI Entertainment,The Green,Hawai'i '13,The Green,Good One,US4CL1310030,886444083997,US,North America,Stream Premium,"20,052",$92.28,$83.05,0RVuF9ekXb1mih0exuh4ZQ,NaN,0.881,0.573,2.0,-8.067,1.0,0.1050,0.04540,0.000000,0.1280,0.925,130.025,206000.0,4.0,NaN,NaN,NaN,track
10001,2021-08,Pandora,MRI Entertainment,The Green,Hawai'i '13,The Green,Power in the Words,US4CL1310031,886444083997,US,North America,Stream Premium,"21,435",$84.94,$76.45,7HIV5qR08KfnTL3lWazcwa,NaN,0.744,0.756,10.0,-8.107,0.0,0.0427,0.06870,0.000004,0.1170,0.906,87.992,278333.0,4.0,NaN,NaN,NaN,track
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4749947,2021-08,TikTok,MRI Entertainment,The Green,Hawai'i '13,The Green,Hawai'i Aloha,US4CL1310040,886444083997,KY,North America,Stream Ad-Supported,1,$0.00,$0.00,4qyyWB9ah6Rd6eMmRsR26q,NaN,0.193,0.467,11.0,-14.405,1.0,0.0458,0.79600,0.000000,0.4430,0.314,112.450,110431.0,3.0,NaN,NaN,NaN,track
4749986,2021-08,TikTok,MRI Entertainment,The Green,Hawai'i '13,The Green,Hawai'i Aloha,US4CL1310040,886444083997,KH,Asia,Stream Ad-Supported,1,$0.00,$0.00,4qyyWB9ah6Rd6eMmRsR26q,NaN,0.193,0.467,11.0,-14.405,1.0,0.0458,0.79600,0.000000,0.4430,0.314,112.450,110431.0,3.0,NaN,NaN,NaN,track
4754204,2021-08,Spotify,MRI Entertainment,The Green,Hawai'i '13,The Green,Good One,US4CL1310030,886444083997,US,North America,Other,1,$0.00,$0.00,0RVuF9ekXb1mih0exuh4ZQ,NaN,0.881,0.573,2.0,-8.067,1.0,0.1050,0.04540,0.000000,0.1280,0.925,130.025,206000.0,4.0,NaN,NaN,NaN,track
4754574,2021-08,Spotify,MRI Entertainment,The Green,Hawai'i '13,The Green,Power in the Words,US4CL1310031,886444083997,CL,South America,Other,3,$0.00,$0.00,7HIV5qR08KfnTL3lWazcwa,NaN,0.744,0.756,10.0,-8.107,0.0,0.0427,0.06870,0.000004,0.1170,0.906,87.992,278333.0,4.0,NaN,NaN,NaN,track


In [22]:
tracks_data['Statement Month'].max()

'2021-08'

In [20]:
tracks_data['Statement Month'].value_counts()

2020-06    10680
2021-05    10337
2020-12     9945
2020-02     9926
2019-09     7675
2019-01     7431
2021-08     6398
2020-05     6228
2020-11     6079
2020-08     5901
2021-04     5742
2020-03     5738
2020-10     5657
2020-09     5637
2020-01     5608
2020-07     5580
2020-04     5576
2021-06     5444
2021-07     5406
2021-02     5396
2021-03     5206
2019-08     5186
2021-01     5180
2019-10     5070
2019-11     4953
2019-07     4852
2019-05     4843
2018-11     4808
2019-02     4754
2018-08     4740
2019-12     4652
2019-06     4589
2018-09     4477
2019-04     4467
2019-03     4455
2018-12     4374
2018-10     4283
2018-05     4172
2018-07     4069
2018-01     3998
2018-04     3981
2018-06     3923
2018-02     3350
2018-03     3181
Name: Statement Month, dtype: int64

In [21]:
tracks_data['Store Name'].value_counts()

iTunes/Apple                                      118874
Spotify                                            58017
Facebook                                           24118
YouTube Subscription                                7600
YouTube                                             7260
Google Play                                         6643
Deezer                                              5943
Amazon Unlimited                                    2608
Amazon Music                                        2229
TIDAL                                               2116
Pandora                                             1546
TikTok                                              1426
SoundCloud Go                                       1183
Napster                                              872
iMusica                                              371
iHeartRadio (Reporting Only)                         359
FreshPlanet / SongPop                                349
AMI Entertainment              